In [2]:
import openai
import re
import requests
import sys
import os
from dotenv import load_dotenv
load_dotenv()

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_key = os.getenv("OPENAI_API_KEY")
openai.api_base = os.getenv("OPENAI_API_BASE")

CHAT_COMPLETIONS_MODEL = os.getenv('GPT_4_DEPLOYMENT')


In [3]:
#Code Generation
prompt = "### Postgres SQL tables, with their properties:\n#\n# Employee(id, name, department_id)\n# Department(id, name, address)\n# Salary_Payments(id, employee_id, amount, date)\n#\n### A query to list the names of the departments which employed more than 10 employees in the last 3 months\n\n query: "
print(prompt)
response = openai.ChatCompletion.create(
  engine=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": prompt}],
  temperature=0,
  max_tokens=150,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])

### Postgres SQL tables, with their properties:
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employee_id, amount, date)
#
### A query to list the names of the departments which employed more than 10 employees in the last 3 months

 query: 
```sql
SELECT d.name
FROM Department d
WHERE (
  SELECT COUNT(DISTINCT e.id)
  FROM Employee e
  JOIN Salary_Payments sp ON e.id = sp.employee_id
  WHERE e.department_id = d.id AND sp.date >= (CURRENT_DATE - INTERVAL '3 months')
) > 10


In [4]:
# Code Explanation
code = "Can you explain what does this code do?\n#\n# ###\n\
   Code:\n\
   SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10\n#\n#\
   Answer:\n# "

response = openai.ChatCompletion.create(
  engine=CHAT_COMPLETIONS_MODEL,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content": code}],
  temperature=0,
  max_tokens=500,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=["#",";"])
print(response['choices'][0]['message']['content'])


This SQL code is used to select the names of departments from a database where more than 10 employees have received salary payments in the last three months.

Here's a breakdown of what the code does:

- `SELECT d.name FROM Department d`: This selects the name of each department from the Department table. The "d" is an alias for the Department table, which makes the code easier to read.

- `JOIN Employee e ON d.id = e.department_id`: This joins the Department table with the Employee table based on the condition that the department's id in the Department table matches the department_id in the Employee table. The "e" is an alias for the Employee table.

- `WHERE e.id IN (SELECT employee_id FROM Salary_Payments WHERE date > now() - interval '3 months')`: This filters the joined table to only include employees who have received salary payments in the last three months. It does this by checking if the employee's id is in the list of employee_ids in the Salary_Payments table where the paymen